## Setup

## Loading the Data

In [ ]:
#Todo

## Adding visualizations 

In [ ]:
#Todo

## preprocessing and Feature engineering

In [ ]:
#Todo

## splitting the dataset

In [ ]:
#Todo

## training the model

In [ ]:
#Todo

## evaluating the model

In [ ]:
#Todo

## Improving the Model

In [ ]:
#Todo